In [ ]:
# Import libraries
from datetime import datetime
import glob
import numpy as np
import matplotlib.pyplot as plt
import cmocean
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
import xskillscore as xs
import hvplot.xarray
import metpy.calc as mpcalc
import metpy.plots as mpplots
from matplotlib.patheffects import withStroke
from metpy.io import parse_metar_file
from metpy.units import pandas_dataframe_to_unit_arrays
from siphon.catalog import TDSCatalog
import re
import dask
import gc
from dask.distributed import Client

In [150]:
from datetime import datetime

In [151]:
# Axis extents for each region in [W, E, S, N] format
# Timezones in hours +- GMT
settings = {
    # Central America (mostly Honduras-Nicaragua-Costa Rica)
    "ca": {"extent": [-91, -81, 7, 17], "tz": -6},
    # South America (mostly eastern Brazil)
    "sa": {"extent": [-65, -30, -15, 0], "tz": -3},
    # Western Australia (mostly near the west coast)
    "wa": {"extent": [113, 123, -35, -30], "tz": +8}
}

In [176]:
# months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun",
#           "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
# years = range(1981, 2022)
avhrr_earliest = "Jan-1981"
modis_earliest = "Mar-2000"
avhrr_latest = "Dec-2018"
modis_latest = "Dec-2021"
subsets = ["all", "djf", "mam", "jja", "son",
           "Jan", "Feb", "Mar", "Apr", "May", "Jun",
           "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

In [177]:
def data_source_to_use(periodstart, periodend):
    if ((periodstart >= datetime.strptime(avhrr_earliest, "%b-%Y")) & 
        (periodstart < datetime.strptime(modis_earliest, "%b-%Y")) &
        (periodend <= datetime.strptime(avhrr_latest, "%b-%Y"))):
        data_source = "avhrr"
    elif ((periodstart >= datetime.strptime(modis_earliest, "%b-%Y")) &
          (periodend <= datetime.strptime(modis_latest, "%b-%Y"))):
        data_source = "modis"
    else:
        raise Exception("If periodstart is before Mar-2000, " +
                        "periodend cannot be after Dec-2018 " +
                        "(since this would cover both an " +
                        "AVHRR-only and a MODIS-only period)")
    return(data_source)

In [195]:
def calc_mlai_climatology(region, periodstart, periodend, subset, outputname=None):
    """
    Calculate mean leaf area index (MLAI) climatology
    
    Args:
        region (str): Region to perform calculation over.
                        Must be one of ["ca", "sa", "wa"]
        periodstart (str): Start of period to perform calculation over.
                        Must be of form "%b-%Y" eg. "Jul-1990".
                        Must be between "Jan-1981" and "Dec-2021".
        periodend (str): End of period to perform calculation over.
                        Must be of form "%b-%Y" eg. "Jul-1990".
                        Must be between "Jan-1981" and "Dec-2021".
        subset (str): Subset of period to perform calculation over.
                        Must be one of ["all", "djf", "mam", "jja", "son",
                        "Jan", "Feb", "Mar", "Apr", "May", "Jun",
                        "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
        outputname (str): Name of output netcdf4 file. If none is
                        given then the default will be
                        region_periodstart_periodend_subset_mlai.nc
    
    For each grid cell, calculate the mean leaf area index (MLAI) over 
    the period from periodstart to periodend. The calculation uses 8-day 
    satellite HDF data from the data_raw folder as input, then outputs 
    the result as a netcdf4 file into the data_processed folder.
    """
    assert region in settings.keys(), \
        f"region must be one of: {*[*settings],}"
    # assert(isinstance(periodstart, str)), {"periodstart must be a string " +
    #                                       "of form '%b-%Y' eg. Jul-1990"}
    # assert(len(periodstart)==8), {"periodstart must be a string " +
    #                               "of form '%b-%Y' eg. Jul-1990"}
    # assert(periodstart[3]=="-"), {"periodstart must be a string " +
    #                               "of form '%b-%Y' eg. Jul-1990"}
    # assert(periodstart[0:3] in months), {"periodstart must be a string " +
    #                                      "of form '%b-%Y' eg. Jul-1990"}
    # assert(int(periodstart[4:8]) in years), {"periodstart must be a string " +
    #                                          "of form '%b-%Y' eg. Jul-1990"}
    periodstart = datetime.strptime(periodstart, "%b-%Y")
    periodend = datetime.strptime(periodend, "%b-%Y")
    assert periodend >= periodstart, \
        "periodend must be equal to or later than periodstart"
    assert periodstart >= datetime.strptime(avhrr_earliest, "%b-%Y"), \
        f"periodstart must be equal to or later than {avhrr_earliest}"
    assert periodend <= datetime.strptime(modis_latest, "%b-%Y"), \
        f"periodend must be equal to or earlier than {modis_latest}"
    data_source = data_source_to_use(periodstart, periodend)
    assert subset in subsets, \
        f"subset must be one of: {*[*subsets],}"
    if outputname:
        assert(isinstance(outputname, str))
    else:
        outputname = f"{region}_{periodstart}_{periodend}_{subset}_mlai_{data_source}"
    
    
    
    print(data_source)
    print(region)

In [196]:
calc_mlai_climatology("ca", "Jul-2000", "Aug-2021", "jja")

modis
ca


In [87]:
datetime.strptime("Feb-1980", "%b-%Y") >= datetime.strptime("Feb-1980", "%b-%Y")

True

In [142]:
True & False

False

In [187]:
isinstance(None, str)

False

In [194]:
if None:
    print("hello")
else:
    print("bye")

bye


In [10]:
areas["wa"]["extent"]

[113, 123, -35, -30]

In [ ]:
# Axis extents for each region in [W, E, S, N] format
extent = {
    # Central America (mostly Honduras-Nicaragua-Costa Rica)
    "ca": [-91, -81, 7, 17],
    # South America (mostly eastern Brazil)
    "sa": [-65, -30, -15, 0],
    # Western Australia (mostly near the west coast)
    "wa": [113, 123, -35, -30]
}

In [ ]:
tz = {"ca": -6, # +- 0
      "sa": -3, # +- 1.5
      "wa": +8} # +- 0